# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [17]:
# your code here

orders = pd.read_csv('Orders.csv')

In [3]:
orders.head(10)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
5,5,536365,22752,2010,12,3,8,set 7 babushka nesting boxes,2,2010-12-01 08:26:00,7.65,17850,United Kingdom,15.30
6,6,536365,21730,2010,12,3,8,glass star frosted t-light holder,6,2010-12-01 08:26:00,4.25,17850,United Kingdom,25.50
7,7,536366,22633,2010,12,3,8,hand warmer union jack,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,11.10
8,8,536366,22632,2010,12,3,8,hand warmer red polka dot,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,11.10
9,9,536367,84879,2010,12,3,8,assorted colour bird ornament,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,54.08


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [26]:
# your code here
# first i need to see the limits of the disired quantiles

quantile_100 = orders['amount_spent'].quantile(q=1)
quantile_95 = orders['amount_spent'].quantile(q=0.95)
quantile_75 = orders['amount_spent'].quantile(q=0.75)


# now that i know the limits, i can cut my dataframe into these categories, preferred customers and vip

orders['Customers Categorie'] = pd.cut(orders['amount_spent'],[0,quantile_75,quantile_95,quantile_100], labels = ['Normal Customer', 'Preferred Customer','VIP Customer'])
orders['Customers Categorie'].value_counts()

Normal Customer       308262
Preferred Customer     70007
VIP Customer           19615
Name: Customers Categorie, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [38]:
# your code here

orders_dummies = pd.get_dummies(orders, columns=['Customers Categorie'])
orders_sum = orders_dummies.groupby('Country').sum()
customers = orders_sum[['Customers Categorie_Normal Customer','Customers Categorie_Preferred Customer','Customers Categorie_VIP Customer']]
customers.sort_values('Customers Categorie_VIP Customer', ascending=False)

,Customers Categorie_Normal Customer,Customers Categorie_Preferred Customer,Customers Categorie_VIP Customer
Country,,,
United Kingdom,281377.0,58205.0,14739.0
Netherlands,432.0,459.0,1468.0
EIRE,4574.0,1947.0,715.0
Australia,339.0,272.0,571.0
Germany,6147.0,2382.0,511.0
France,5605.0,2343.0,393.0
Switzerland,1070.0,592.0,179.0
Sweden,181.0,93.0,177.0
Japan,80.0,69.0,172.0


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [39]:
# your code here

customers['VIP+Preferred'] = customers['Customers Categorie_Preferred Customer'] + customers['Customers Categorie_VIP Customer']
customers.sort_values('VIP+Preferred',ascending=False)

<ipython-input-39-1cc5f288e2a5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers['VIP+Preferred'] = customers['Customers Categorie_Preferred Customer'] + customers['Customers Categorie_VIP Customer']


,Customers Categorie_Normal Customer,Customers Categorie_Preferred Customer,Customers Categorie_VIP Customer,VIP+Preferred
Country,,,,
United Kingdom,281377.0,58205.0,14739.0,72944.0
Germany,6147.0,2382.0,511.0,2893.0
France,5605.0,2343.0,393.0,2736.0
EIRE,4574.0,1947.0,715.0,2662.0
Netherlands,432.0,459.0,1468.0,1927.0
Australia,339.0,272.0,571.0,843.0
Switzerland,1070.0,592.0,179.0,771.0
Spain,1885.0,470.0,129.0,599.0
Belgium,1450.0,542.0,39.0,581.0
